In [1]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
#from calcRMSE import calcRMSE
import numpy as np
from getConfig import getConfig

In [2]:
h2o.no_progress()
h2o.init(port=54323)

Checking whether there is an H2O instance running at http://localhost:54323. connected.


H2O cluster uptime:,9 days 7 hours 21 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,11 days
H2O cluster name:,tbrownex
H2O cluster total nodes:,1
H2O cluster free memory:,827 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"locked, healthy"


In [3]:
def getData(config):
    typ = {"Pool": "string"}
    return h2o.import_file(path = config["dataLoc"] + config["fileName"], col_types=typ)

In [4]:
# These fields are useless in predicting:
# "Job", "Year", "Work Site", "BD Status", "Major EAC", "Income Statement Type", 
# "Calc Type", "Revenue Calc Type", "Auto Earn Adj", "Award Fee" 
# The list holds the index values of these columns
def dropCols(df):
    df = df.drop([0, 1, 11, 16, 20, 21, 22, 23, 24, 26])
    return df

In [5]:
def buildModel(parms):
    numTrees = parms[0]
    leafSize = parms[1]
    return H2ORandomForestEstimator(
        model_id="rf_v1",
        ntrees=numTrees,
        min_rows=leafSize,
        stopping_rounds=4)

In [6]:
def showVariables(model):
    results = model._model_json['output']['variable_importances'].as_data_frame()
    pct = results["scaled_importance"].round(2)
    var = results["variable"]
    tom = zip(var, pct)
    for x in tom:
        print(x)

In [7]:
def evaluate(test, preds):
    preds = preds.as_data_frame().as_matrix()
    results = test[["Pool", "Actual"]].as_data_frame()
    results["Pred"] = preds
    # These 3 are 86% of the total spend
    pools = ["10", "18", "30", "6S", "20"]
    results = results.loc[results["Pool"].isin(pools)]
    results.set_index("Pool", inplace=True)
    pool = results.groupby(level=0)

    score = 0
    for x in pool:
        pred = x[1]["Pred"].sum()
        act = x[1]["Actual"].sum()
        acc = abs(pred - act) / act
        score += acc
    score = score/len(pool)
    return score

In [8]:
def process(p):
    model = buildModel(p)
    model.train(features, y=label, training_frame=train, validation_frame=valid)
    #showVariables(model)
    preds = model.predict(test)
    score = evaluate(test, preds)
    trees = p[0]
    leaf = p[1]
    rec = str(trees) +","+ str(leaf) +","+ str(score)+"\n"
    #scores.write(rec)
    print(p, ": ", score)
    test["Prediction"] = preds
    h2o.export_file(test, "/home/tbrownex/full.csv", force=True)

In [9]:
'''def evaluate(test, preds):
    preds = preds.as_data_frame().as_matrix()
    np.savetxt("/home/tbrownex/predictions.csv", preds, delimiter=",")
    actual = test["Actual"].as_data_frame().as_matrix()
    np.savetxt("/home/tbrownex/actual.csv", actual, delimiter=",")
    return calcRMSE(actual, preds)'''

'def evaluate(test, preds):\n    preds = preds.as_data_frame().as_matrix()\n    np.savetxt("/home/tbrownex/predictions.csv", preds, delimiter=",")\n    actual = test["Actual"].as_data_frame().as_matrix()\n    np.savetxt("/home/tbrownex/actual.csv", actual, delimiter=",")\n    return calcRMSE(actual, preds)'

In [10]:
scores = open("/home/tbrownex/scores.csv", "w")

In [16]:
config = getConfig()
df = getData(config)

In [17]:
# Drop useless columns
df = dropCols(df)

In [18]:
df = df.drop("Plan")

###### Test to see how valuable these are

In [19]:
'''df = df.drop("Revenues")
df = df.drop("Allowable Cost")
df = df.drop("Total Orders")'''

'df = df.drop("Revenues")\ndf = df.drop("Allowable Cost")\ndf = df.drop("Total Orders")'

In [20]:
# Last column is "Actual", what we're trying to predict
features = df.col_names[:-1]      
label    = df.col_names[-1]    

In [23]:
'''parms = []
numTrees = [100*x for x in range(1,25)]
leafSize = [5]
for t in numTrees:
    for s in leafSize:
        parm = (t,s)
        parms.append(parm)
'''
numTrees = 20000
leafSize = 5
parm = (numTrees, leafSize)
parms=[parm]

loops=1
for n in range(loops):
    for p in parms:
        train, valid, test = df.split_frame([0.6, 0.2])
        process(p)

scores.close()

/home/tbrownex/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(20000, 5) :  0.23540788677192012
